In [1]:
%run func_def.ipynb

imports loaded.

Pre-Processing func:
********************
    get_height(item)
    get_weight(item)
    get_reach(item)
    get_std_reach(item, height, reach_list, height_list)
    get_age(item)
    get_std_age(item, age_list)
    get_win_perc(name, date, df)
    get_lose_streak_data_frame(name, date, df)
    get_win_streak_data_frame(name, date, df)
    get_streak_data_frame(name, date, df)
    get_win_streak(name, date, df)
    get_lose_streak(name, date, df)
    get_fighter_stats(name, data )
    add_columns(data, column_list)

Feature Engineering func:
*************************
    get_stat(index, data, stat='height', fo='')
    get_swaped_row(index, data)
    get_wins(combo, data)
    get_wins_ratio(combo, data)

stats_model_training func:
********************************
    get_sig_features(data, sig_r)
    get_r_score(X, y)
    set_train_test(X_columns, y_columns, data, test_size=0.3, valid_size=0.3)
    set_random_forest_reg(X_columns, y_columns, data)
    set_predictions(mod

In [2]:
#%run stats_model_training.ipynb

In [3]:
#%run result_model_training.ipynb

In [4]:
#%run method_model_training.ipynb

### <span style='color:blue'> Insert new fight: </span> 

In [5]:
fighter = 'Kleydson Rodrigues'
opponent = 'Shannon Ross'
print(fighter + ' vs ' + opponent)

Kleydson Rodrigues vs Shannon Ross


### Fight Preview(load statistics data frame):

In [6]:
loc = '../data/'
file = 'ufc_fighters_stats_processed.csv'
stats = pd.read_csv(loc+file)
stats = stats[stats.columns[1:]]
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

,Kleydson Rodrigues,Shannon Ross
wins,7,13
losses,0.0,6.0
draws,-0,0
height,167.64,167.64
weight,56.25,56.25
reach,170.18,171.75
stance,Orthodox,Switch
age,27.0,33.0
slpm,5.73,7.15
str. acc.,0.61,0.38


### load main data frame:

In [7]:
path = '../data/fight_stat_data_trained.csv'
df = pd.read_csv(path)
df = df[df.columns[0:len(df.columns)-4]]
df.date = pd.to_datetime(df.date)
print('Data Frame loaded: '+str(df.shape))
df.tail(3)

Data Frame loaded: (6829, 22)


,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
6826,validation,2017-07-29,Kailin Curran,Aleksandra Albu,lose,U-DEC:,-0.00892,0.123128,-1.597639,0.008337,-0.150322,0.248259,-0.545754,0.199566,-0.613584,-0.320786,-0.491150,1.249440,-0.054481,-2.040434,-0.589427,2.164739
6827,test,2017-07-29,Jarred Brooks,Eric Shelton,win,S-DEC:,-0.00892,-0.176583,1.561212,0.008337,0.950517,0.213119,0.767752,0.391150,-0.305805,0.329644,-0.129731,-0.099707,1.584492,0.005137,-0.017747,-1.062286
6828,train,2017-07-29,Drew Dober,Joshua Burkman,win,KO/TKO:Punch,1.34476,0.123128,-0.018213,0.008337,-1.012090,-0.665378,-0.579434,-0.183603,1.094906,-0.239482,0.783004,-0.015385,1.671822,0.132985,-0.017747,-2.137961


In [8]:
match = set_new_fight(fighter=fighter, opponent=opponent, data=df)
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-28,None,Kleydson Rodrigues,Shannon Ross,None,None,None,None,1.0,0.25,-0.34,1.0,-1.42,0.23,-5.36,-0.06,-0.003398,0,0,1


### Load fight statistics prediction model:

In [9]:
stat_pred_reg = load_trained_model(filename_path='../global_variables/stats_pred_random_forest_reg.pkl')
X = stat_pred_reg.feature_names_in_
y = ['kd_dif','str_dif','td_dif','sub_dif']
feature_importance = stat_pred_reg.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/stats_pred_random_forest_reg.pkl loaded.


,feature_importances
td avg._dif,0.198445
slpm_dif,0.158586
sapm_dif,0.129302
sub. avg._dif,0.121630
td def._dif,0.074261
td acc._dif,0.060929
str. acc._dif,0.060838
str. def_dif,0.057333
win_perc_dif,0.052760
win_strk_dif,0.041707


#### Load predictions to match:

In [10]:
stats_pred_list = stat_pred_reg.predict(match[stat_pred_reg.feature_names_in_])

i = 0
for stat in ['kd_dif','str_dif','td_dif','sub_dif']:
    match.loc[0,stat] = stats_pred_list[0][i]
    i += 1

match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-28,None,Kleydson Rodrigues,Shannon Ross,-0.203275,0.133619,0.686111,0.339132,1.0,0.25,-0.34,1.0,-1.42,0.23,-5.36,-0.06,-0.003398,0,0,1


### <span style='color:green'> Statistics prediction to the match: </span>

In [11]:
match[y]

,kd_dif,str_dif,td_dif,sub_dif
0,-0.203275,0.133619,0.686111,0.339132


### Load fight result prediction model:

In [12]:
result_pred_clf = load_trained_model(filename_path='../global_variables/result_pred_random_forest_clf.pkl')
X = result_pred_clf.feature_names_in_
y = 'result'
feature_importance = result_pred_clf.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/result_pred_random_forest_clf.pkl loaded.


,feature_importances
str_dif,0.342261
kd_dif,0.117279
td_dif,0.088850
sub_dif,0.081213
slpm_dif,0.050975
td def._dif,0.044409
td avg._dif,0.039579
sapm_dif,0.038602
str. def_dif,0.033760
td acc._dif,0.032881


In [13]:
result_pred_list = result_pred_clf.predict(match[result_pred_clf.feature_names_in_])
match.loc[0,y] = result_pred_list[0]
winner = get_winner(match=match) #saving winner before 'result' becoming a feature
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-28,win,Kleydson Rodrigues,Shannon Ross,-0.203275,0.133619,0.686111,0.339132,1.0,0.25,-0.34,1.0,-1.42,0.23,-5.36,-0.06,-0.003398,0,0,1


### <span style='color:green'> Result prediction to the match(fighter win/lose): </span>

In [14]:
print(match.loc[0,y])

win


### Load method of winning prediction model:

In [15]:
method_pred_clf = load_trained_model(filename_path='../global_variables/method_pred_random_forest_clf.pkl')
X = method_pred_clf.feature_names_in_
y = 'method'
feature_importance = method_pred_clf.feature_importances_

fi = pd.DataFrame(data=feature_importance, index=X, columns=['feature_importances']).sort_values(by=['feature_importances'], ascending=False)
fi

../global_variables/method_pred_random_forest_clf.pkl loaded.


,feature_importances
sub_dif,0.232966
kd_dif,0.151420
str_dif,0.121370
td_dif,0.098519
sub. avg._dif,0.053991
td avg._dif,0.050612
slpm_dif,0.047561
td def._dif,0.044536
sapm_dif,0.043714
td acc._dif,0.042690


#### converting 'result' to feature:

In [16]:
df[df.fighter==fighter]

,set,date,fighter,opponent,result,method,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
978,test,2022-05-07,Kleydson Rodrigues,CJ Vergara,lose,S-DEC:,-0.00892,0.347911,0.771499,0.858138,0.555772,0.880777,-0.242637,0.965903,0.542159,0.410948,-0.950581,-0.26835,-0.054481,0.005137,-0.017747,-1.062286


In [17]:
result = match.loc[0,'result']
match.loc[0,'result'] = 0 

if len(df[df.fighter==fighter]) > 0:
    match.loc[0,'result'] = len(df[(df.result==result)&(df.fighter==fighter)])/len(df[df.fighter==fighter])
    
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,None,2023-02-28,0.0,Kleydson Rodrigues,Shannon Ross,-0.203275,0.133619,0.686111,0.339132,1.0,0.25,-0.34,1.0,-1.42,0.23,-5.36,-0.06,-0.003398,0,0,1


In [18]:
method_pred_list = method_pred_clf.predict(match[method_pred_clf.feature_names_in_])
match.loc[0,y] = method_pred_list[0]
method = match.loc[0,'method']
match

,set,method,date,result,fighter,opponent,kd_dif,str_dif,td_dif,sub_dif,td avg._dif,td acc._dif,td def._dif,sub. avg._dif,slpm_dif,str. acc._dif,sapm_dif,str. def_dif,stance_win_ratio,win_perc_dif,win_strk_dif,lose_strk_dif
0,None,decision,2023-02-28,0.0,Kleydson Rodrigues,Shannon Ross,-0.203275,0.133619,0.686111,0.339132,1.0,0.25,-0.34,1.0,-1.42,0.23,-5.36,-0.06,-0.003398,0,0,1


In [19]:
gp = pd.read_csv('../global_variables/ml_models_parameters.csv')

result_accuracy = -1
method_accuracy = -1
for model in gp.ml_model:
    if 'result_' in model:
        result_accuracy = gp.loc[gp.ml_model == model,'accuracy']
    if 'method_' in model:
        method_accuracy = gp.loc[gp.ml_model == model,'accuracy']    

### <span style='color:green'> Match predictions summary: </span>

In [20]:
print('--------------------------------------')
print('%s vs %s:' %(fighter, opponent))
print('--------------------------------------')
print('Winner: %s [Accuracy = %.3f%%]' %(winner,100*result_accuracy))
print('By: %s [Accuracy = %.3f%%]' %(method,100*method_accuracy))
print('--------------------------------------')
get_fighters_stats(fighter=fighter, opponent=opponent, data=stats)

--------------------------------------
Kleydson Rodrigues vs Shannon Ross:
--------------------------------------
Winner: Kleydson Rodrigues [Accuracy = 86.877%]
By: decision [Accuracy = 72.132%]
--------------------------------------


,Kleydson Rodrigues,Shannon Ross
wins,7,13
losses,0.0,6.0
draws,-0,0
height,167.64,167.64
weight,56.25,56.25
reach,170.18,171.75
stance,Orthodox,Switch
age,27.0,33.0
slpm,5.73,7.15
str. acc.,0.61,0.38
